<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-2" data-toc-modified-id="Chapter-2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chapter 2</a></span></li></ul></div>

## Chapter 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from urllib.request import urlretrieve
from io import StringIO

In [2]:
file = 'lanl_flows.csv'
url = 'https://assets.datacamp.com/production/repositories/3554/datasets/18a574bfeef99241c2fe45db6314fdaeb4b288fe/'
urlretrieve(url+file, file)
flows = pd.read_csv(file)
flows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
time                    10000 non-null int64
duration                10000 non-null int64
source_computer         10000 non-null object
source_port             10000 non-null object
destination_computer    10000 non-null object
destination_port        10000 non-null object
protocol                10000 non-null int64
packet_count            10000 non-null int64
byte_count              10000 non-null int64
dtypes: int64(5), object(4)
memory usage: 703.2+ KB


In [11]:
file = 'redteam.csv'
url = 'https://assets.datacamp.com/production/repositories/3554/datasets/e52ee3093aee49f8599dd30dc0acada35cbc2873/'
urlretrieve(url+file, file)
attacks = pd.read_csv(file)
attacks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
time                    748 non-null int64
user_domain             748 non-null object
source_computer         748 non-null object
destination_computer    748 non-null object
dtypes: int64(1), object(3)
memory usage: 23.5+ KB


In [25]:
bads = set(attacks['source_computer'])
bads.update(set(attacks['destination_computer']))
len(bads)

305

In [26]:
def featurize(df):
    return {
        'unique_ports': len(set(df['destination_port'])),
        'average_packet': np.mean(df['packet_count']),
        'average_duration': np.mean(df['duration'])
    }

In [27]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [33]:
# Group by source computer, and apply the feature extractor
out = flows.groupby('source_computer').apply(featurize)

# Convert the iterator to a dataframe by calling list on it
X = pd.DataFrame(list(out), index = out.index)

# Check which sources in X.index are bad to create labels
y = [x in bads for x in X.index]

# Report the average accuracy of Adaboost over 3-fold CV
print(np.mean(cross_val_score(AdaBoostClassifier(), X, y)))

/Users/Brian/anaconda3/envs/datacamp/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.9361199939089387


In [42]:
# Create a feature counting unique protocols per source
protocols = flows.groupby('source_computer').apply(
  lambda df: df['protocol'].nunique())

# Convert this feature into a dataframe, naming the column
protocols_DF = pd.DataFrame(
  protocols, index=protocols.index, columns=['protocol'])

# Now concatenate this feature with the previous dataset, X
X_more = pd.concat([X, protocols_DF], axis=1)

# Refit the classifier and report its accuracy
print(np.mean(cross_val_score(
  AdaBoostClassifier(), X_more, y)))

/Users/Brian/anaconda3/envs/datacamp/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.9377950357849856


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [82]:
# Create a new dataset X_train_bad by subselecting bad hosts
X_train_bad = X_train[y_train]

# Calculate the average of unique_ports in bad examples
avg_bad_ports = np.mean(X_train_bad['unique_ports'])

# Label as positive sources that use more ports than that
pred_port = X_test['unique_ports'] > avg_bad_ports

# Print the accuracy of the heuristic
print(accuracy_score(y_test, pred_port))

0.9395973154362416


In [83]:
# Compute the mean of average_packet for bad sources
avg_bad_packet = np.mean(X_train[y_train]['average_packet'])

# Label as positive if average_packet is lower than that
pred_packet = X_test['average_packet'] < avg_bad_packet

# Find indices where pred_port and pred_packet both True
pred_port = X_test['unique_ports'] > avg_bad_ports
pred_both = pred_packet & pred_port

# Ports only produced an accuracy of 0.919. Is this better?
print(accuracy_score(y_test, pred_both))

0.9463087248322147


In [108]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

y_train_noisy = y_train.copy()
y_train_noisy[:100] = [True] * 100
np.sum(y_train_noisy)

(121, False)

In [103]:
# Fit a Gaussian Naive Bayes classifier to the training data
clf = GaussianNB().fit(X_train, y_train_noisy)

# Report its accuracy on the test data
print(accuracy_score(y_test, clf.predict(X_test)))

# Assign half the weight to the first 100 noisy examples
weights = [0.5]*100 + [1.0]*(len(y_train)-100)

# Refit using weights and report accuracy. Has it improved?
clf_weights = GaussianNB().fit(X_train, y_train_noisy, sample_weight=weights)
print(accuracy_score(y_test, clf_weights.predict(X_test)))

0.09395973154362416
0.09395973154362416


In [148]:
from sklearn.metrics import f1_score, precision_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
preds = clf_weights.predict(X_test)

In [153]:
tp, fp, fn, tn = confusion_matrix(y_test, preds).ravel()
print(f1_score(preds, y_test))
print(precision_score(preds, y_test))
print((tp + tn)/len(y_test))

0.28571428571428575
0.18181818181818182
0.9328859060402684


In [154]:
# Fit a random forest classifier to the training data
clf = RandomForestClassifier(random_state=2).fit(X_train, y_train)

# Label the test data
preds = clf.predict(X_test)

# Get false positives/negatives from the confusion matrix
tp, fp, fn, tn = confusion_matrix(y_test, preds).ravel()

# Now compute the cost using the manager's advice
cost = fp*10000 + fn*150000

/Users/Brian/anaconda3/envs/datacamp/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [155]:
cost

1360000

In [169]:
tp = 168
fp = 19
fn = 49
tn = 25

precision = tp/(tp+fp)
recall = tp/(tp+fn)

In [170]:
precision, recall

(0.8983957219251337, 0.7741935483870968)

In [171]:
flows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
time                    10000 non-null int64
duration                10000 non-null int64
source_computer         10000 non-null object
source_port             10000 non-null object
destination_computer    10000 non-null object
destination_port        10000 non-null object
protocol                10000 non-null int64
packet_count            10000 non-null int64
byte_count              10000 non-null int64
dtypes: int64(5), object(4)
memory usage: 703.2+ KB


In [183]:
X_str = ',duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate\n0,0,1,11,9,295,12884,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n1,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n2,0,1,11,9,227,2662,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n3,0,1,11,9,246,255,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,51,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n4,0,1,11,9,283,22227,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,255,1.0,0.0,0.05,0.04,0.0,0.0,0.0,0.0\n5,51,1,24,9,1169,332,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,59,164,0.81,0.07,0.02,0.01,0.0,0.0,0.0,0.0\n6,0,1,11,9,307,7902,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,255,1.0,0.0,0.2,0.03,0.0,0.02,0.0,0.0\n7,0,1,11,9,247,15693,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n8,0,1,11,9,313,285,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n9,0,1,11,9,176,2087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n10,0,1,11,9,220,253,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n11,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n12,0,1,11,9,294,4227,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n13,0,2,4,9,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,221,43,0.17,0.03,0.17,0.05,0.0,0.0,0.0,0.0\n14,0,1,11,9,242,449,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n15,0,1,11,9,363,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,166,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n16,0,1,11,9,314,1542,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,62,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n17,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n18,0,1,11,9,214,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,255,1.0,0.0,0.11,0.02,0.0,0.0,0.0,0.0\n19,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n20,0,1,11,9,203,247,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,25,0.0,0.0,0.0,0.0,1.0,0.0,0.12,93,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n21,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n22,0,1,11,9,200,2536,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,167,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n23,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,4,0.0,0.0,0.2,0.25,0.8,0.4,0.0,74,74,1.0,0.0,0.01,0.0,0.0,0.0,0.04,0.04\n24,0,1,11,9,203,573,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n25,0,1,11,9,214,805,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n26,0,1,7,9,9,141,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,237,12,0.05,0.02,0.0,0.17,0.0,0.0,0.0,0.0\n27,0,1,11,9,342,3530,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,42,255,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n28,0,1,11,9,285,10053,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20,255,1.0,0.0,0.05,0.04,0.0,0.0,0.0,0.0\n29,0,1,11,9,223,9558,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,23,255,1.0,0.0,0.04,0.06,0.0,0.0,0.0,0.0\n30,0,1,11,9,302,455,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,13,0.0,0.0,0.0,0.0,1.0,0.0,0.15,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n31,0,1,11,9,220,1456,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,19,0.0,0.0,0.0,0.0,1.0,0.0,0.16,68,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n32,0,1,11,9,344,4838,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,39,0.0,0.0,0.0,0.0,1.0,0.0,0.05,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n33,0,1,11,9,340,2873,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n34,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n35,0,1,11,9,230,2970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,17,0.0,0.0,0.0,0.0,1.0,0.0,0.24,15,255,1.0,0.0,0.07,0.04,0.0,0.0,0.0,0.0\n36,0,1,11,9,319,1161,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,30,0.0,0.0,0.0,0.0,1.0,0.0,0.13,167,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n37,0,1,11,9,249,8946,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,157,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n38,0,1,11,9,150,10263,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,34,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n39,0,2,18,9,48,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,4,0.8,0.4,0.8,0.0,0.0,0.0,0.0,0.0\n40,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n41,0,2,4,9,33,89,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5,0.0,0.0,0.0,0.0,1.0,0.0,0.8,15,11,0.73,0.13,0.07,0.0,0.0,0.0,0.0,0.0\n42,0,1,11,9,222,771,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n43,0,1,11,9,210,5939,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n44,0,1,11,9,304,691,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,119,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n45,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n46,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n47,0,1,11,9,218,7274,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,255,1.0,0.0,0.09,0.03,0.0,0.0,0.0,0.0\n48,0,1,11,9,312,1344,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n49,0,1,11,9,371,487,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,33,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n50,0,1,11,9,201,1856,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n51,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n52,0,1,11,9,151,9551,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,37,255,1.0,0.0,0.03,0.02,0.0,0.0,0.0,0.0\n53,0,1,11,9,226,10440,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,255,1.0,0.0,0.09,0.01,0.0,0.0,0.0,0.0\n54,0,1,11,9,250,6944,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,30,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n55,0,1,9,9,7321,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,176,82,0.18,0.02,0.18,0.02,0.0,0.0,0.01,0.0\n56,0,1,11,9,292,2367,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53,255,1.0,0.0,0.02,0.02,0.02,0.0,0.0,0.0\n57,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n58,0,1,11,9,236,1281,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,34,0.0,0.0,0.0,0.0,1.0,0.0,0.12,212,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n59,0,1,11,9,251,306,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,98,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n60,0,1,11,9,197,412,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n61,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n62,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n63,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n64,0,1,11,9,313,173,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,20,0.0,0.0,0.0,0.0,1.0,0.0,0.15,3,255,1.0,0.0,0.33,0.04,0.0,0.0,0.0,0.0\n65,0,2,4,9,45,134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,134,0.0,0.0,0.0,0.0,1.0,0.0,0.01,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n66,0,1,11,9,220,261,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,111,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n67,0,1,11,9,225,1303,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,107,255,1.0,0.0,0.01,0.01,0.0,0.01,0.0,0.0\n68,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n69,0,1,11,9,209,9921,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,144,255,1.0,0.0,0.01,0.02,0.01,0.0,0.0,0.0\n70,0,1,11,9,189,2543,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n71,0,1,11,9,212,425,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,26,255,1.0,0.0,0.04,0.01,0.0,0.0,0.0,0.0\n72,0,1,11,9,330,1548,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13,255,1.0,0.0,0.08,0.04,0.0,0.0,0.0,0.0\n73,0,1,11,9,220,1109,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,7,0.0,0.0,0.0,0.0,1.0,0.0,0.43,46,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n74,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n75,0,1,11,9,209,362,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n76,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n77,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n78,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n79,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n80,0,1,11,9,235,2015,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n81,3,1,11,9,260,193,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.05,0.0,0.0,0.0,0.0\n82,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n83,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n84,0,1,11,9,243,513,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,19,255,1.0,0.0,0.05,0.03,0.0,0.0,0.0,0.0\n85,0,1,11,9,228,1451,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n86,0,1,11,9,300,12164,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,25,25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n87,0,1,11,9,247,397,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n88,0,1,9,9,1690,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.0,0.0,0.0,96,91,0.33,0.03,0.33,0.02,0.01,0.01,0.0,0.0\n89,0,1,11,9,245,2110,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,255,1.0,0.0,0.33,0.05,0.0,0.0,0.0,0.0\n90,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.0,0.17,1.0,0.0,0.33,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.03,0.03\n91,0,1,11,9,310,1461,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n92,0,1,11,9,224,3609,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,7,255,1.0,0.0,0.14,0.05,0.0,0.0,0.0,0.0\n93,0,1,11,9,359,280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5,255,1.0,0.0,0.2,0.01,0.0,0.0,0.0,0.0\n94,0,1,11,9,221,294,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n95,0,1,11,9,258,6491,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,131,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n96,0,1,11,9,337,1109,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n97,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n98,0,1,11,9,309,909,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n99,0,1,11,9,211,309,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12,255,1.0,0.0,0.08,0.01,0.0,0.0,0.0,0.0\n100,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n101,1,1,24,9,1093,331,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,123,110,0.5,0.02,0.01,0.02,0.0,0.0,0.0,0.0\n102,0,1,11,9,285,347,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,238,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n103,0,1,11,9,225,431,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n104,0,1,11,9,214,620,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,3,255,1.0,0.0,0.33,0.04,0.0,0.0,0.0,0.0\n105,0,1,11,9,220,309,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n106,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n107,0,1,11,9,152,38363,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,22,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n108,0,1,11,9,261,3931,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,109,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n109,0,1,11,9,290,1701,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n110,0,1,11,9,219,302,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,16,0.0,0.0,0.0,0.0,1.0,0.0,0.19,17,255,1.0,0.0,0.06,0.09,0.0,0.0,0.0,0.0\n111,0,1,11,9,281,1646,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,65,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n112,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n113,0,1,11,9,328,8315,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,5,255,1.0,0.0,0.2,0.05,0.0,0.0,0.0,0.0\n114,2,1,27,9,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18,2,0.06,1.0,0.06,1.0,0.0,0.0,0.89,0.0\n115,0,1,11,9,187,5545,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n116,0,2,4,9,35,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,174,157,0.9,0.02,0.01,0.0,0.0,0.0,0.03,0.0\n117,0,1,11,9,211,6245,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n118,0,1,11,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1.0,0.5,0.0,0.0,1.0,0.0,1.0,1,255,1.0,0.0,1.0,0.04,1.0,0.0,0.0,0.0\n119,0,1,24,9,682,325,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,64,177,0.69,0.06,0.02,0.01,0.0,0.0,0.0,0.0\n120,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n121,0,2,4,9,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,197,41,0.16,0.03,0.16,0.05,0.0,0.0,0.0,0.0\n122,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,15,255,1.0,0.0,0.07,0.1,0.0,0.0,1.0,1.0\n123,0,1,24,9,856,335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,117,0.33,0.67,0.33,0.03,0.0,0.0,0.0,0.0\n124,0,1,11,9,160,25837,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,255,1.0,0.0,0.14,0.04,0.0,0.0,0.0,0.0\n125,0,1,11,9,363,398,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n126,0,1,11,9,237,275,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n127,0,1,11,9,225,1548,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n128,0,2,4,9,40,93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,71,228,1.0,0.0,1.0,0.01,0.0,0.0,0.0,0.0\n129,0,1,11,9,282,2957,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,13,0.0,0.0,0.0,0.0,1.0,0.0,0.15,174,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n130,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n131,0,1,11,9,332,1541,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,96,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n132,0,1,11,6,292,12335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,20,0.14,0.05,0.0,0.0,1.0,0.0,0.2,43,255,1.0,0.0,0.02,0.01,0.02,0.0,0.0,0.0\n133,0,1,11,9,247,2402,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n134,0,1,11,9,312,1227,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n135,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n136,0,1,21,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,255,1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0\n137,0,1,11,9,389,11894,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,8,255,1.0,0.0,0.12,0.03,0.0,0.0,0.0,0.0\n138,0,0,5,9,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2,255,1.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0\n139,0,1,11,9,203,277,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,15,255,1.0,0.0,0.07,0.07,0.0,0.0,0.0,0.0\n140,0,1,11,9,319,255,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,202,255,1.0,0.0,0.0,0.02,0.01,0.01,0.0,0.0\n141,0,1,24,9,1001,327,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,215,166,0.77,0.02,0.01,0.0,0.0,0.0,0.0,0.0\n142,0,1,11,9,346,1127,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,17,0.0,0.0,0.0,0.0,1.0,0.0,0.24,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n143,0,1,24,9,861,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,70,178,0.84,0.04,0.01,0.02,0.0,0.01,0.01,0.0\n144,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,176,176,1.0,0.0,0.01,0.0,0.0,0.0,0.06,0.06\n145,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n146,0,1,11,9,315,3049,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,24,0.0,0.0,0.0,0.0,1.0,0.0,0.08,2,124,1.0,0.0,0.5,0.06,0.0,0.0,0.0,0.0\n147,0,1,11,9,309,923,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,24,255,1.0,0.0,0.04,0.05,0.0,0.0,0.0,0.0\n148,0,1,11,9,324,3410,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n149,0,1,11,9,209,280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,237,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n150,3,2,4,9,53,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25,18,0.72,0.08,0.64,0.0,0.0,0.0,0.0,0.0\n151,0,1,11,9,285,5790,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n152,0,1,11,9,286,1108,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,79,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n153,0,1,9,9,8737,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,63,0.25,0.02,0.25,0.0,0.0,0.0,0.0,0.0\n154,0,1,11,9,250,7886,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n155,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n156,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n157,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n158,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n159,0,1,11,9,302,6542,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,53,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n160,0,1,11,9,248,18992,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,110,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n161,0,1,11,9,342,3583,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,67,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0\n162,0,1,11,9,289,4520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n163,0,1,11,9,239,1423,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,125,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n164,0,1,11,9,289,6847,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17,255,1.0,0.0,0.06,0.03,0.0,0.0,0.0,0.0\n165,0,1,11,9,217,1726,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n166,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n167,0,1,11,9,282,6355,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n168,0,1,24,9,1189,325,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,84,246,0.96,0.02,0.01,0.01,0.0,0.0,0.0,0.0\n169,0,1,11,9,218,2816,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,98,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n170,0,1,11,9,215,5341,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,36,0.0,0.0,0.0,0.0,1.0,0.0,0.08,50,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n171,0,1,11,9,288,3084,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,191,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n172,0,1,11,9,177,3158,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n173,0,1,11,9,231,4288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,31,0.0,0.0,0.0,0.0,1.0,0.0,0.16,9,255,1.0,0.0,0.11,0.03,0.0,0.0,0.0,0.0\n174,0,1,11,9,312,422,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n175,0,1,9,9,12324,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53,12,0.11,0.11,0.11,0.17,0.02,0.0,0.0,0.0\n176,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n177,0,1,11,9,286,323,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,24,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n178,0,1,11,9,224,1390,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n179,0,1,11,9,258,917,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,32,32,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n180,0,1,11,9,282,624,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,47,157,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n181,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n182,0,1,11,9,306,950,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,38,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n183,28,1,8,9,943,2720,0,0,0,19,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,101,22,0.22,0.04,0.01,0.0,0.0,0.0,0.0,0.0\n184,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n185,0,1,11,9,227,4027,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21,255,1.0,0.0,0.05,0.04,0.0,0.0,0.0,0.0\n186,0,1,11,9,239,34084,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10,255,1.0,0.0,0.1,0.05,0.0,0.0,0.0,0.0\n187,0,1,11,9,319,390,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n188,0,1,11,9,322,920,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,57,255,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n189,0,1,11,9,220,370,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n190,0,1,11,9,214,25519,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n191,1,1,24,9,1088,328,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7,244,0.86,0.29,0.14,0.01,0.0,0.0,0.0,0.0\n192,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11,255,1.0,0.0,0.09,0.24,0.0,0.0,1.0,0.95\n193,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n194,0,1,11,9,307,1075,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n195,0,1,11,9,329,299,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,39,255,1.0,0.0,0.03,0.01,0.0,0.0,0.0,0.0\n196,0,1,11,9,296,2531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,188,188,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n197,0,1,11,9,162,13187,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,238,238,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n198,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.04,0.04\n199,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n200,0,1,11,9,164,16963,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n201,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n202,0,1,11,9,316,5050,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,114,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n203,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n204,0,1,11,9,207,626,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,30,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n205,0,1,11,9,219,3081,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n206,0,1,11,9,243,3092,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n207,0,1,11,9,315,515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n208,0,1,11,9,337,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,55,255,1.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0\n209,0,1,11,9,290,7318,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23,255,1.0,0.0,0.04,0.05,0.0,0.0,0.0,0.0\n210,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n211,0,1,11,9,284,2081,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n212,0,1,11,9,201,392,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,44,0.0,0.0,0.0,0.0,1.0,0.0,0.09,230,230,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n213,1,1,24,9,1222,335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,77,238,0.97,0.03,0.01,0.01,0.0,0.0,0.0,0.0\n214,0,1,11,9,193,327,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,28,0.0,0.0,0.0,0.0,1.0,0.0,0.07,27,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n215,0,1,11,9,320,272,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,6,1.0,0.0,0.17,0.0,0.0,0.0,0.0,0.0\n216,0,1,11,9,324,268,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,205,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n217,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n218,0,1,11,9,199,1195,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,36,255,1.0,0.0,0.03,0.02,0.0,0.0,0.0,0.0\n219,0,1,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,7,2,0.14,1.0,0.14,1.0,0.0,0.0,0.86,1.0\n220,1,1,24,9,2219,340,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,255,100,0.39,0.02,0.0,0.0,0.0,0.0,0.0,0.0\n221,0,1,11,9,223,1706,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,37,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n222,0,1,11,9,218,1211,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,80,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n223,0,1,11,9,217,2139,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,34,0.0,0.0,0.0,0.0,1.0,0.0,0.06,33,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n224,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,1.0,0.33,1.0,0.0,0.67,18,255,1.0,0.0,0.06,0.17,0.0,0.0,1.0,1.0\n225,0,1,11,9,205,594,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,98,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n226,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n227,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n228,0,1,11,9,197,4980,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n229,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n230,0,1,11,9,309,646,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,37,255,1.0,0.0,0.03,0.01,0.0,0.01,0.0,0.0\n231,0,1,11,9,328,3960,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,160,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n232,0,1,11,9,284,414,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n233,0,1,11,9,343,783,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,6,0.0,0.0,0.0,0.0,1.0,0.0,0.5,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n234,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n235,0,1,11,9,169,180,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n236,0,1,11,9,353,1423,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n237,0,1,11,9,284,16493,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n238,0,1,11,9,226,6356,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n239,0,1,24,9,1183,334,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,68,139,0.38,0.1,0.01,0.02,0.0,0.0,0.0,0.0\n240,0,1,11,9,205,873,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,75,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n241,0,1,11,9,494,5324,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,18,0.0,0.0,0.0,0.0,1.0,0.0,0.17,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n242,0,2,4,9,44,110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,118,208,1.0,0.0,1.0,0.01,0.0,0.0,0.0,0.0\n243,0,1,11,9,228,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,230,0.9,0.02,0.0,0.0,0.0,0.0,0.0,0.0\n244,0,1,11,9,295,2057,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16,255,1.0,0.0,0.06,0.02,0.0,0.0,0.0,0.0\n245,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n246,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n247,0,1,9,9,383,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,230,61,0.27,0.03,0.27,0.0,0.0,0.0,0.0,0.0\n248,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n249,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n250,0,1,11,9,178,14496,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n251,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n252,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,62,62,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n253,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n254,0,1,11,9,321,22780,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,22,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n255,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n256,0,1,11,9,229,353,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,22,0.0,0.0,0.0,0.0,1.0,0.0,0.14,13,255,1.0,0.0,0.08,0.05,0.0,0.0,0.0,0.0\n257,0,2,4,9,44,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,234,234,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,227,0.89,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n258,0,1,11,9,237,8734,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,21,0.0,0.0,0.0,0.0,1.0,0.0,0.1,8,255,1.0,0.0,0.12,0.01,0.0,0.0,0.0,0.0\n259,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n260,0,1,11,9,255,3324,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21,255,1.0,0.0,0.05,0.02,0.0,0.0,0.0,0.0\n261,0,1,11,9,187,312,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,19,0.0,0.0,0.0,0.0,1.0,0.0,0.16,223,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n262,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n263,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n264,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,497,497,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n265,0,1,11,9,297,1645,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n266,0,1,11,9,336,1304,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n267,0,1,11,9,246,1588,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,47,255,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n268,0,1,11,9,366,279,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,102,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n269,0,1,11,9,293,43825,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,35,35,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n270,0,1,11,9,355,1483,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n271,0,1,11,9,213,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,26,26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,159,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n272,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n273,0,1,11,9,234,3222,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.25,1.0,0.0,0.5,255,242,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n274,0,1,11,9,274,2290,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,54,0.0,0.0,0.0,0.0,1.0,0.0,0.07,72,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n275,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n276,0,1,11,9,196,6107,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20,255,1.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0\n277,0,1,11,9,372,421,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,11,0.0,0.0,0.0,0.0,1.0,0.0,0.27,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n278,0,1,11,9,350,1112,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,8,0.0,0.0,0.0,0.0,1.0,0.0,0.38,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n279,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n280,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n281,0,1,11,9,234,520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,35,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n282,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n283,0,1,11,9,248,7588,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,255,1.0,0.0,0.17,0.04,0.0,0.01,0.0,0.0\n284,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n285,0,1,11,9,222,16404,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,33,0.0,0.0,0.0,0.0,1.0,0.0,0.06,18,255,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0\n286,0,1,11,9,193,294,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,10,255,1.0,0.0,0.1,0.04,0.0,0.0,0.0,0.0\n287,0,1,11,9,151,120,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,255,1.0,0.0,0.25,0.05,0.0,0.0,0.0,0.0\n288,0,1,11,9,246,11204,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12,255,1.0,0.0,0.08,0.03,0.0,0.0,0.0,0.0\n289,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n290,0,1,11,9,296,784,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n291,0,1,11,9,261,240,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,246,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n292,1,1,24,9,1120,421,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,50,244,0.98,0.04,0.02,0.01,0.0,0.0,0.0,0.0\n293,0,1,11,9,225,1155,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,8,0.0,0.0,0.0,0.0,1.0,0.0,0.38,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n294,0,1,11,9,250,2066,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n295,0,1,11,9,206,365,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,219,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n296,0,1,11,9,217,2073,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,25,25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n297,45,1,26,9,466,1514273,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,184,57,0.31,0.02,0.01,0.0,0.0,0.0,0.0,0.0\n298,0,1,11,9,230,4600,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18,255,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n299,0,1,9,9,245,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,142,0.56,0.03,0.56,0.0,0.0,0.01,0.0,0.0\n300,0,1,11,9,246,3085,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,65,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n301,0,1,11,9,232,19485,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n302,0,1,11,9,287,264,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,50,0.0,0.0,0.0,0.0,1.0,0.0,0.08,220,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n303,0,1,11,9,167,6722,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n304,0,1,11,9,169,10105,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.06,0.0,0.0,0.0,0.0\n305,3,1,24,9,42759,324,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,75,183,0.47,0.05,0.01,0.02,0.0,0.0,0.0,0.0\n306,0,2,4,9,36,101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30,20,0.67,0.1,0.03,0.0,0.0,0.0,0.0,0.0\n307,0,1,11,9,325,10757,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,35,0.0,0.0,0.0,0.0,1.0,0.0,0.14,35,255,1.0,0.0,0.03,0.02,0.0,0.0,0.0,0.0\n308,0,1,11,9,210,2274,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,55,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n309,0,1,11,9,294,2497,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,255,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n310,0,1,0,9,10,38,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,95,5,0.05,0.06,0.01,0.0,0.0,0.0,0.0,0.0\n311,0,1,11,9,248,1374,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,124,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0\n312,0,1,11,9,313,660,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,140,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n313,0,1,11,9,310,1478,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10,255,1.0,0.0,0.1,0.02,0.0,0.0,0.0,0.0\n314,0,1,11,9,186,10243,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n315,0,2,4,9,44,135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61,139,0.0,0.0,0.0,0.0,1.0,0.0,0.01,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n316,0,1,11,9,204,1389,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,77,79,0.0,0.0,0.0,0.0,1.0,0.0,0.03,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n317,0,1,9,9,1874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,223,42,0.17,0.02,0.17,0.05,0.0,0.0,0.0,0.0\n318,0,1,11,9,197,3889,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,255,250,0.98,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n319,0,1,11,9,221,16096,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,103,226,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n320,0,1,11,9,156,944,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n321,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n322,0,1,11,9,283,397,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n323,0,1,11,9,213,810,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n324,0,1,11,9,223,7936,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,46,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n325,0,1,11,9,353,4988,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,28,255,1.0,0.0,0.04,0.05,0.0,0.0,0.0,0.0\n326,0,1,11,9,199,765,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,39,39,0.0,0.0,0.0,0.0,1.0,0.0,0.0,121,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n327,0,1,11,9,294,5231,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,41,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n328,0,1,11,9,208,1314,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45,255,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n329,0,1,11,9,199,2055,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,147,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n330,0,1,11,9,189,2543,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,115,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n331,0,1,11,9,269,745,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,205,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n332,0,1,11,9,208,4894,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n333,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n334,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n335,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n336,0,1,11,9,281,611,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,228,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n337,0,1,11,9,290,802,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,21,0.0,0.0,0.0,0.0,1.0,0.0,0.1,62,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n338,1,1,24,9,1039,447,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,67,246,0.96,0.03,0.01,0.01,0.0,0.0,0.0,0.0\n339,0,1,11,9,317,742,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,229,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n340,0,1,9,9,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,148,43,0.29,0.04,0.29,0.0,0.0,0.0,0.0,0.0\n341,0,1,11,9,212,21953,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,20,0.0,0.0,0.0,0.0,1.0,0.0,0.1,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n342,0,1,11,9,236,35870,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,173,255,1.0,0.0,0.01,0.02,0.01,0.0,0.0,0.0\n343,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,48,48,1.0,0.0,0.02,0.0,0.0,0.0,0.04,0.04\n344,0,1,11,9,174,1788,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n345,0,1,11,9,285,2345,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,255,1.0,0.0,0.33,0.01,0.0,0.0,0.0,0.0\n346,0,1,11,9,223,887,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n347,0,1,11,9,235,309,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n348,0,1,11,9,235,11485,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n349,0,1,11,9,210,758,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,21,0.0,0.0,0.0,0.0,1.0,0.0,0.14,113,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n350,0,1,11,9,319,397,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,79,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n351,0,1,11,9,361,7396,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,232,255,1.0,0.0,0.0,0.02,0.01,0.01,0.0,0.0\n352,0,1,11,9,333,2132,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n353,0,1,11,9,331,298,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,22,22,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n354,0,1,11,9,232,2866,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,28,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n355,0,1,11,9,223,703,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n356,0,1,11,9,351,306,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,30,0.0,0.0,0.0,0.0,1.0,0.0,0.13,41,255,1.0,0.0,0.02,0.02,0.0,0.01,0.0,0.0\n357,0,1,11,9,278,11756,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n358,0,1,11,9,252,855,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,255,1.0,0.0,0.14,0.06,0.0,0.0,0.0,0.0\n359,0,1,11,9,216,605,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n360,0,1,11,9,334,3200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n361,0,1,9,9,768,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,33,64,0.61,0.06,0.61,0.05,0.0,0.0,0.0,0.0\n362,0,1,11,9,216,339,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,76,76,0.0,0.0,0.0,0.0,1.0,0.0,0.0,159,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n363,0,1,11,9,235,30113,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n364,0,1,11,9,294,998,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n365,0,1,11,9,215,4937,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,35,35,0.0,0.0,0.0,0.0,1.0,0.0,0.0,200,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n366,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n367,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n368,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n369,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n370,0,1,11,9,212,2251,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,35,38,0.0,0.0,0.0,0.0,1.0,0.0,0.08,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n371,0,1,11,9,192,10429,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,255,1.0,0.0,0.12,0.02,0.0,0.0,0.0,0.0\n372,0,1,11,9,216,303,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,15,0.0,0.0,0.0,0.0,1.0,0.0,0.13,62,63,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n373,0,1,11,9,323,3675,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,36,0.0,0.0,0.0,0.0,1.0,0.0,0.08,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n374,0,1,11,9,221,2999,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n375,0,1,11,9,166,34720,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,173,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n376,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n377,0,1,11,9,345,3437,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,32,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n378,0,1,11,9,229,4869,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n379,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n380,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n381,0,1,11,9,238,1378,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,16,0.0,0.0,0.0,0.0,1.0,0.0,0.31,17,255,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0\n382,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n383,0,1,24,9,1055,335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,4,0.8,0.4,0.2,0.0,0.0,0.0,0.0,0.0\n384,0,1,11,9,235,328,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n385,0,1,11,9,278,10994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,89,209,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0\n386,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n387,0,1,11,9,249,737,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,105,239,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n388,0,1,9,9,12241,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53,61,0.81,0.04,0.81,0.03,0.0,0.0,0.0,0.0\n389,0,1,11,9,341,2753,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,131,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n390,1,1,24,9,1511,327,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,54,171,0.39,0.09,0.02,0.02,0.0,0.0,0.0,0.0\n391,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n392,0,1,11,9,248,9850,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25,255,1.0,0.0,0.04,0.05,0.0,0.0,0.0,0.0\n393,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n394,0,1,24,9,957,330,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,53,168,0.58,0.09,0.02,0.01,0.0,0.0,0.0,0.0\n395,0,1,11,9,199,644,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,255,1.0,0.0,0.11,0.09,0.0,0.0,0.0,0.0\n396,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n397,0,1,11,9,199,665,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,12,255,1.0,0.0,0.08,0.05,0.0,0.0,0.0,0.0\n398,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n399,0,1,11,9,260,618,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,24,0.0,0.0,0.0,0.0,1.0,0.0,0.08,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n400,0,2,4,9,45,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,54,0.0,0.0,0.0,0.0,1.0,0.0,0.0,210,170,0.81,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n401,0,1,11,9,244,5036,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.0,0.0,172,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n402,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n403,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n404,0,1,11,9,222,11821,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n405,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n406,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n407,0,1,11,9,202,4620,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,28,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n408,6,1,24,9,1468,326,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,63,98,0.22,0.03,0.02,0.02,0.0,0.0,0.0,0.0\n409,63,2,18,9,48,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,22,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n410,0,1,11,9,231,506,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,34,55,0.0,0.0,0.0,0.0,1.0,0.0,0.13,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n411,0,1,11,9,304,1042,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,6,0.0,0.0,0.0,0.0,1.0,0.0,0.67,3,255,1.0,0.0,0.33,0.02,0.0,0.0,0.0,0.0\n412,0,1,11,9,229,3168,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,84,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n413,0,1,11,9,329,267,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n414,0,1,11,9,298,2510,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31,255,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n415,0,1,11,9,217,2227,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,15,0.0,0.0,0.0,0.0,1.0,0.0,0.2,30,214,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n416,0,1,11,9,289,280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,78,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n417,0,1,11,9,268,31108,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,202,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n418,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n419,0,1,11,9,217,738,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n420,0,1,11,9,245,20264,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,7,0.0,0.0,0.0,0.0,1.0,0.0,0.43,52,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n421,0,1,9,9,2191,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,77,119,0.34,0.06,0.34,0.02,0.0,0.0,0.0,0.0\n422,0,1,11,9,273,316,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,17,0.0,0.0,0.0,0.0,1.0,0.0,0.18,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n423,0,1,11,9,223,741,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n424,0,1,24,9,1745,367,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,23,248,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n425,0,1,11,9,323,584,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,25,0.0,0.0,0.0,0.0,1.0,0.0,0.16,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n426,0,1,11,9,238,1280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,35,0.0,0.0,0.0,0.0,1.0,0.0,0.11,96,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n427,0,1,11,9,223,2955,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,255,1.0,0.0,0.05,0.08,0.0,0.0,0.0,0.0\n428,0,1,11,9,506,2419,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n429,0,1,11,9,230,3432,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n430,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n431,0,1,11,9,220,7829,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n432,0,1,11,9,224,1356,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,7,101,1.0,0.0,0.14,0.09,0.0,0.01,0.0,0.0\n433,0,1,11,9,214,537,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,64,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n434,0,1,11,9,340,931,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n435,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1,15,1.0,0.0,1.0,0.33,0.0,0.0,1.0,1.0\n436,0,1,11,9,310,2171,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,19,0.0,0.0,0.0,0.0,1.0,0.0,0.16,188,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n437,0,1,24,9,1551,332,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,11,64,1.0,0.0,0.09,0.05,0.0,0.0,0.0,0.0\n438,0,1,11,9,316,1526,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n439,0,1,11,9,193,800,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n440,0,1,11,9,339,3360,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n441,0,1,11,9,219,11487,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,25,255,1.0,0.0,0.04,0.07,0.0,0.0,0.0,0.0\n442,0,1,11,9,325,1724,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n443,0,1,11,9,321,9329,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,21,0.0,0.0,0.0,0.0,1.0,0.0,0.19,46,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n444,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n445,0,1,11,9,203,1046,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,65,255,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n446,0,1,11,9,295,1087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,47,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n447,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n448,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n449,0,1,11,9,231,442,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,148,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n450,0,1,11,9,253,1969,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n451,0,1,11,9,238,306,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,43,43,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n452,0,1,11,9,285,272,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,34,255,1.0,0.0,0.03,0.06,0.0,0.0,0.0,0.0\n453,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.03,0.03\n454,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n455,0,2,4,9,44,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,250,210,0.84,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n456,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n457,0,1,11,9,183,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,22,255,1.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0\n458,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,58,255,1.0,0.0,0.02,0.09,0.0,0.0,1.0,1.0\n459,0,1,9,9,190,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,27,6,0.22,0.07,0.22,0.0,0.0,0.0,0.0,0.0\n460,0,1,11,9,170,11520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,23,0.0,0.0,0.0,0.0,1.0,0.0,0.09,25,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n461,0,1,11,9,297,5260,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,32,0.0,0.0,0.0,0.0,1.0,0.0,0.06,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n462,0,1,11,9,223,1273,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,12,0.0,0.0,0.0,0.0,1.0,0.0,0.17,21,255,1.0,0.0,0.05,0.06,0.0,0.0,0.0,0.0\n463,0,1,11,9,318,13954,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,11,0.0,0.0,0.0,0.0,1.0,0.0,0.18,17,206,1.0,0.0,0.06,0.08,0.0,0.0,0.0,0.0\n464,6,1,24,9,1145,366,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,192,136,0.52,0.03,0.01,0.01,0.0,0.0,0.0,0.0\n465,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n466,0,1,11,9,235,327,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,34,0.0,0.0,0.0,0.0,1.0,0.0,0.12,125,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n467,0,1,11,9,304,74221,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,24,0.0,0.0,0.0,0.0,1.0,0.0,0.12,3,255,1.0,0.0,0.33,0.05,0.0,0.0,0.0,0.0\n468,0,1,11,9,305,1075,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,243,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n469,0,1,11,9,306,2728,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,144,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n470,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.01,0.01,0.04,0.04\n471,0,1,11,9,226,392,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n472,0,1,11,9,510,6471,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,8,0.0,0.0,0.0,0.0,1.0,0.0,0.38,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n473,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n474,0,1,11,9,221,2367,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,185,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n475,0,1,11,9,276,749,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,33,0.0,0.0,0.0,0.0,1.0,0.0,0.09,18,255,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n476,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n477,0,1,11,9,209,1199,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,43,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n478,0,1,11,9,318,6534,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n479,0,1,21,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,2,0.81,1.0,0.19,0.0,0.05,0.41,0.0,255,2,0.01,0.06,0.15,0.0,0.12,1.0,0.03,0.0\n480,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n481,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,11,255,1.0,0.0,0.09,0.31,0.0,0.0,1.0,0.94\n482,0,1,11,9,325,2538,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,35,35,0.0,0.0,0.0,0.0,1.0,0.0,0.0,190,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n483,0,1,11,9,268,6746,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,4,255,1.0,0.0,0.25,0.03,0.0,0.0,0.0,0.0\n484,0,1,9,9,172,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,28,0.7,0.08,0.7,0.0,0.0,0.0,0.0,0.0\n485,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n486,0,1,11,9,275,4112,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n487,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n488,0,1,11,9,273,7024,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,175,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n489,0,1,11,9,249,2840,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,27,0.0,0.0,0.0,0.0,1.0,0.0,0.07,168,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n490,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n491,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,13,255,1.0,0.0,0.08,0.18,0.0,0.0,1.0,0.98\n492,0,1,11,9,229,12884,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n493,0,1,11,9,304,312,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,255,1.0,0.0,0.25,0.05,0.0,0.0,0.0,0.0\n494,0,1,9,9,854,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,97,0.16,0.06,0.16,0.02,0.0,0.0,0.0,0.0\n495,0,1,11,9,220,695,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,22,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n496,0,1,11,9,203,405,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,8,255,1.0,0.0,0.12,0.06,0.0,0.0,0.0,0.0\n497,0,1,11,9,220,574,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n498,0,1,11,9,215,29375,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,164,255,1.0,0.0,0.01,0.01,0.01,0.0,0.0,0.0\n499,0,1,11,9,313,2078,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n500,0,1,11,9,300,2208,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,79,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n501,0,1,11,9,309,2020,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21,255,1.0,0.0,0.05,0.03,0.0,0.0,0.0,0.0\n502,0,1,11,9,289,1542,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18,255,1.0,0.0,0.06,0.02,0.0,0.0,0.0,0.0\n503,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,172,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n504,0,1,11,9,291,1205,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,12,0.0,0.0,0.0,0.0,1.0,0.0,0.17,227,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n505,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n506,0,1,11,9,339,3360,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n507,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n508,0,1,11,9,291,1812,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,255,1.0,0.0,0.12,0.07,0.0,0.0,0.0,0.0\n509,0,1,11,9,203,7662,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n510,0,1,11,9,380,1596,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n511,0,1,11,9,295,16432,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,120,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n512,0,1,11,9,333,287,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n513,0,1,11,9,250,409,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n514,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n515,0,1,11,9,295,1529,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,42,0.0,0.0,0.0,0.0,1.0,0.0,0.05,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n516,0,2,4,9,44,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,255,243,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n517,0,2,4,9,44,73,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,169,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,227,0.89,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n518,0,1,11,9,304,21281,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,23,255,1.0,0.0,0.04,0.01,0.0,0.0,0.0,0.0\n519,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n520,0,1,11,9,283,17995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,122,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n521,0,1,24,9,747,332,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,41,246,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n522,0,1,11,9,297,3867,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n523,0,1,11,9,312,1176,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.0,0.0,24,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n524,0,1,11,9,213,1071,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,26,0.0,0.0,0.0,0.0,1.0,0.0,0.08,100,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n525,0,1,11,9,227,573,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n526,0,1,11,9,218,290,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n527,0,1,11,9,214,2870,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n528,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.03,0.03\n529,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n530,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n531,0,1,11,9,220,909,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10,255,1.0,0.0,0.1,0.05,0.0,0.0,0.0,0.0\n532,0,1,11,9,232,255,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,38,38,0.0,0.0,0.0,0.0,1.0,0.0,0.0,38,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n533,0,1,11,9,301,4407,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,16,201,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n534,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n535,0,1,11,9,242,1408,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,24,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n536,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n537,0,1,11,9,365,240,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,156,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n538,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,1.0,1.0,1.0,0.0,1.0,2,255,1.0,0.0,0.5,0.07,0.0,0.0,1.0,0.93\n539,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n540,0,1,11,9,309,1075,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,242,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n541,0,1,11,9,206,2171,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n542,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n543,0,1,11,9,284,810,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.43,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n544,0,1,11,9,217,1730,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,20,0.0,0.0,0.0,0.0,1.0,0.0,0.15,98,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n545,0,1,11,9,253,455,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n546,0,1,11,9,183,2713,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,255,1.0,0.0,0.25,0.03,0.0,0.0,0.0,0.0\n547,0,1,11,9,352,1645,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n548,0,1,11,9,212,3792,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n549,0,1,11,9,230,15818,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n550,0,1,11,9,315,9998,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,255,1.0,0.0,0.2,0.01,0.0,0.0,0.0,0.0\n551,0,1,11,9,255,7782,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,65,199,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n552,0,1,11,9,218,3338,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,52,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n553,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n554,0,1,11,9,233,259,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,22,0.0,0.0,0.0,0.0,1.0,0.0,0.14,52,255,1.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0\n555,0,1,11,9,211,1828,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18,255,1.0,0.0,0.06,0.01,0.0,0.0,0.0,0.0\n556,0,1,11,9,302,2507,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n557,0,1,11,9,284,2286,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n558,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n559,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n560,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n561,0,2,4,9,44,117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n562,0,1,11,9,306,918,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n563,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.27,0.0,0.0,1.0,1.0\n564,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.33,1.0,0.0,0.67,2,2,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0\n565,0,1,11,9,292,28022,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n566,0,1,11,9,288,950,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,125,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n567,0,1,11,9,371,1205,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,31,35,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n568,0,1,11,9,201,762,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n569,0,1,24,9,2464,279,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,184,162,0.55,0.03,0.01,0.01,0.0,0.0,0.0,0.0\n570,0,1,0,9,9,34,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31,4,0.13,0.13,0.03,0.0,0.0,0.0,0.0,0.0\n571,0,2,4,9,43,75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,39,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,253,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n572,0,1,11,9,285,261,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,30,0.0,0.0,0.0,0.0,1.0,0.0,0.13,230,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n573,0,1,11,9,234,405,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,25,30,0.0,0.0,0.0,0.0,1.0,0.0,0.1,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n574,0,1,11,9,243,2281,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,169,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n575,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n576,0,1,11,9,210,633,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13,255,1.0,0.0,0.08,0.04,0.0,0.0,0.0,0.0\n577,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n578,0,1,11,9,319,1259,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n579,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n580,0,1,11,9,317,6329,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n581,0,1,11,9,224,1529,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n582,0,1,11,9,235,299,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,54,255,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n583,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n584,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n585,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n586,0,1,11,9,314,2743,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,255,1.0,0.0,0.5,0.04,0.0,0.0,0.0,0.0\n587,0,1,11,9,222,1461,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,9,0.0,0.0,0.0,0.0,1.0,0.0,0.33,44,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n588,0,1,11,9,336,714,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,15,0.0,0.0,0.0,0.0,1.0,0.0,0.13,183,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n589,0,1,11,9,321,1090,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,255,1.0,0.0,0.11,0.02,0.0,0.0,0.0,0.0\n590,0,1,24,9,630,331,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,161,0.9,0.05,0.03,0.02,0.0,0.0,0.0,0.0\n591,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n592,0,1,11,9,240,2833,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,47,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n593,0,2,4,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,135,16,0.12,0.03,0.12,0.0,0.0,0.0,0.0,0.0\n594,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.04,0.04\n595,0,1,11,9,311,2475,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,167,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n596,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n597,0,1,11,9,306,2507,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,250,0.98,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n598,0,1,11,9,218,17733,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,27,27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n599,0,1,11,9,217,1819,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,30,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n600,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n601,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n602,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3,250,1.0,0.0,0.33,0.15,0.0,0.0,1.0,1.0\n603,0,1,11,9,288,462,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,32,37,0.0,0.0,0.0,0.0,1.0,0.0,0.14,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n604,0,1,11,9,310,1481,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,23,0.0,0.0,0.0,0.0,1.0,0.0,0.17,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n605,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n606,23,1,8,9,333,1054,0,0,0,6,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,124,56,0.45,0.05,0.01,0.0,0.0,0.0,0.0,0.0\n607,0,1,11,9,243,683,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n608,0,1,11,9,193,7355,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,163,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n609,0,1,11,9,339,2076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,21,0.0,0.0,0.0,0.0,1.0,0.0,0.1,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n610,0,1,11,9,301,10352,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,25,0.0,0.0,0.0,0.0,1.0,0.0,0.08,77,187,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n611,0,1,24,9,844,331,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,151,1.0,0.0,0.25,0.02,0.0,0.0,0.0,0.0\n612,0,1,11,9,226,382,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n613,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n614,0,1,11,9,282,2047,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,10,1.0,0.0,0.14,0.2,0.0,0.2,0.0,0.1\n615,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n616,0,1,11,9,296,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,11,0.0,0.0,0.0,0.0,1.0,0.0,0.18,89,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n617,0,1,11,9,218,313,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n618,0,1,11,9,321,1707,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n619,0,1,11,9,162,29122,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n620,0,1,11,9,228,12728,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,255,1.0,0.0,0.11,0.04,0.0,0.0,0.0,0.0\n621,0,1,11,9,226,321,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n622,0,1,11,9,249,280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n623,0,1,24,9,1625,335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,69,104,0.57,0.09,0.01,0.02,0.0,0.0,0.0,0.0\n624,0,1,11,9,317,1726,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,32,0.0,0.0,0.0,0.0,1.0,0.0,0.06,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n625,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.06,0.06\n626,0,1,11,9,205,492,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,56,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n627,0,1,11,9,234,773,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,116,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n628,0,1,11,9,232,1302,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,76,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0\n629,0,1,11,9,213,633,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,156,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n630,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n631,0,1,11,9,271,9248,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,22,0.0,0.0,0.0,0.0,1.0,0.0,0.18,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n632,0,1,11,9,276,1046,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,11,0.0,0.0,0.0,0.0,1.0,0.0,0.27,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n633,0,1,11,9,200,406,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,24,0.0,0.0,0.0,0.0,1.0,0.0,0.08,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n634,0,1,11,9,228,2037,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14,255,1.0,0.0,0.07,0.09,0.0,0.0,0.0,0.0\n635,0,1,11,9,194,1276,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,255,1.0,0.0,0.17,0.02,0.0,0.0,0.0,0.0\n636,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n637,0,1,24,9,1434,366,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,105,95,0.9,0.03,0.01,0.0,0.0,0.0,0.0,0.0\n638,0,1,11,9,326,1511,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n639,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n640,0,1,11,9,262,24845,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n641,1,1,24,9,1947,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,81,163,0.49,0.05,0.01,0.01,0.0,0.0,0.0,0.0\n642,0,1,11,9,238,7222,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n643,0,1,11,9,293,12334,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,19,0.0,0.0,0.0,0.0,1.0,0.0,0.21,126,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n644,0,1,11,9,276,4648,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16,255,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n645,0,1,11,9,200,5537,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,10,255,1.0,0.0,0.1,0.03,0.0,0.0,0.0,0.0\n646,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n647,0,1,11,9,236,2206,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,18,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18,255,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n648,0,1,11,9,201,5949,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,255,1.0,0.0,0.05,0.06,0.0,0.0,0.0,0.0\n649,0,1,11,9,208,836,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,255,1.0,0.0,0.5,0.04,0.0,0.0,0.0,0.0\n650,0,1,11,9,153,364,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,52,0.0,0.0,0.0,0.0,1.0,0.0,0.08,16,255,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0\n651,0,1,11,9,333,6534,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n652,0,1,11,9,267,6592,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6,0.0,0.0,0.0,0.0,1.0,0.0,0.5,1,255,1.0,0.0,1.0,0.04,0.0,0.0,0.0,0.0\n653,0,1,11,9,231,10819,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,255,1.0,0.0,0.05,0.01,0.0,0.0,0.0,0.0\n654,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.17,0.29,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n655,0,1,11,9,199,788,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,25,25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n656,0,2,4,9,44,69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66,66,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n657,0,1,11,9,297,2275,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n658,0,1,11,9,224,7937,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,51,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n659,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n660,0,1,11,9,322,15153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,51,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n661,0,1,11,9,291,31067,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,47,255,1.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0\n662,0,1,24,9,2928,401,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,19,174,0.95,0.11,0.05,0.02,0.0,0.0,0.0,0.0\n663,293,1,11,9,336,276,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,248,0.97,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n664,0,1,11,9,242,513,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,13,255,1.0,0.0,0.08,0.03,0.0,0.0,0.0,0.0\n665,0,1,11,9,312,863,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,130,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n666,0,1,11,9,211,1759,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,54,255,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n667,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n668,0,1,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,14,1,0.07,1.0,0.07,0.0,0.0,0.0,0.86,1.0\n669,0,1,11,9,217,13892,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,29,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n670,0,1,11,9,230,2558,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,65,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n671,0,1,11,9,250,1332,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n672,0,1,11,9,303,4937,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,244,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n673,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n674,0,1,11,9,306,7922,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,3,255,1.0,0.0,0.33,0.01,0.0,0.0,0.0,0.0\n675,0,2,4,9,45,111,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,43,123,0.0,0.0,0.0,0.0,1.0,0.0,0.02,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n676,0,1,11,9,287,12161,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n677,0,1,11,9,207,335,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,58,59,0.0,0.0,0.0,0.0,1.0,0.0,0.03,59,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n678,0,1,11,9,220,2171,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32,255,1.0,0.0,0.03,0.07,0.0,0.0,0.0,0.0\n679,0,1,11,9,324,1158,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n680,0,1,11,9,288,597,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,220,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n681,0,1,11,9,218,1910,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,28,0.0,0.0,0.0,0.0,1.0,0.0,0.14,15,255,1.0,0.0,0.07,0.06,0.0,0.0,0.0,0.0\n682,0,1,11,9,239,2125,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,53,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n683,0,1,24,9,1049,334,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,223,195,0.87,0.02,0.0,0.0,0.0,0.0,0.0,0.0\n684,0,1,11,9,329,7334,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,79,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n685,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n686,0,1,11,9,204,1698,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,36,255,1.0,0.0,0.03,0.02,0.0,0.0,0.0,0.0\n687,0,1,11,9,241,734,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,24,0.0,0.0,0.0,0.0,1.0,0.0,0.08,23,255,1.0,0.0,0.04,0.04,0.0,0.04,0.0,0.0\n688,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n689,0,1,11,9,287,1729,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,24,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n690,0,1,11,9,276,5844,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n691,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n692,0,1,11,9,301,1075,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,250,0.98,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n693,0,1,11,9,185,1526,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n694,0,2,4,9,43,129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n695,0,1,11,9,216,4687,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n696,0,1,11,9,234,710,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n697,0,1,21,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,255,1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0\n698,0,1,11,9,320,22037,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n699,0,1,11,9,310,14185,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,15,0.0,0.0,0.0,0.0,1.0,0.0,0.13,193,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n700,0,1,11,9,218,7384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n701,0,1,11,9,226,9115,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,47,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n702,0,1,11,9,210,5565,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20,255,1.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0\n703,0,1,11,9,324,920,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,13,0.0,0.0,0.0,0.0,1.0,0.0,0.15,33,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n704,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n705,0,1,11,9,183,581,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32,255,1.0,0.0,0.03,0.05,0.0,0.01,0.0,0.0\n706,0,1,11,9,255,3748,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,32,40,0.0,0.0,0.0,0.0,1.0,0.0,0.1,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n707,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n708,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n709,0,2,4,9,46,117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,255,235,0.92,0.01,0.0,0.0,0.0,0.0,0.02,0.0\n710,1,1,24,9,1099,330,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,183,75,0.37,0.02,0.01,0.03,0.01,0.0,0.0,0.0\n711,0,1,9,9,495,0,0,0,0,0,0,1,0,0,0,5,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,119,41,0.34,0.05,0.34,0.0,0.0,0.0,0.0,0.0\n712,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n713,0,1,11,9,296,1875,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n714,0,2,4,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.75,28,4,0.14,0.14,0.14,0.0,0.0,0.0,0.0,0.0\n715,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n716,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n717,0,1,11,9,269,1592,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n718,0,1,11,9,220,2750,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,4,0.0,0.0,0.0,0.0,1.0,0.0,0.75,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n719,0,1,11,9,252,2445,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n720,0,1,11,9,213,1660,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,25,29,0.0,0.0,0.0,0.0,1.0,0.0,0.1,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n721,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n722,0,2,18,9,48,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,82,56,0.68,0.02,0.68,0.0,0.0,0.0,0.0,0.0\n723,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n724,0,1,11,9,168,16350,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n725,0,1,11,9,306,6377,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.01,0.0,0.0,0.0,0.0\n726,0,1,11,9,217,1012,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,57,0.0,0.0,0.0,0.0,1.0,0.0,0.09,18,255,1.0,0.0,0.06,0.04,0.0,0.0,0.0,0.0\n727,0,1,11,9,291,1792,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n728,0,1,11,9,219,572,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n729,0,1,11,9,286,7971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,2,255,1.0,0.0,0.5,0.04,0.0,0.0,0.0,0.0\n730,0,1,9,9,383,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,47,3,0.06,0.09,0.06,0.0,0.0,0.0,0.0,0.0\n731,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n732,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n733,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n734,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n735,0,1,11,9,321,882,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,106,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n736,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n737,0,1,11,9,326,1484,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,22,0.0,0.0,0.0,0.0,1.0,0.0,0.09,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n738,0,1,11,9,389,5029,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n739,0,1,11,9,194,1226,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0.0,0.0,0.0,0.0,1.0,0.0,0.6,9,255,1.0,0.0,0.11,0.05,0.0,0.0,0.0,0.0\n740,0,1,11,9,309,1645,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n741,0,1,11,9,212,1179,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,255,1.0,0.0,0.09,0.03,0.0,0.0,0.0,0.0\n742,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n743,0,1,11,9,252,273525,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,238,0.93,0.02,0.0,0.0,0.0,0.0,0.0,0.0\n744,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n745,0,1,11,9,354,1255,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n746,0,1,11,9,219,787,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n747,0,1,24,9,1185,329,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,154,120,0.78,0.03,0.01,0.0,0.0,0.0,0.0,0.0\n748,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.14,0.25,1.0,0.0,0.25,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n749,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n750,0,1,11,9,217,1511,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,19,0.0,0.0,0.0,0.0,1.0,0.0,0.11,42,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n751,0,1,11,9,221,9388,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,12,0.0,0.0,0.0,0.0,1.0,0.0,0.33,90,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n752,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n753,0,1,11,9,240,3516,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,35,35,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.07,0.0,0.0,0.0,0.0\n754,0,1,11,9,205,1515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30,255,1.0,0.0,0.03,0.04,0.0,0.0,0.0,0.0\n755,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n756,0,1,11,9,292,10420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.08,0.08,0.0,0.0,1.0,0.0,0.0,13,255,1.0,0.0,0.08,0.04,0.08,0.0,0.0,0.0\n757,0,1,24,9,645,360,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,188,0.74,0.02,0.0,0.0,0.0,0.0,0.0,0.0\n758,0,1,9,9,59,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,172,64,0.24,0.41,0.24,0.03,0.0,0.0,0.38,0.02\n759,0,1,24,9,1548,332,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,167,245,0.95,0.02,0.01,0.01,0.0,0.0,0.0,0.0\n760,0,1,11,9,244,347,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n761,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,20,255,1.0,0.0,0.05,0.19,0.0,0.0,1.0,1.0\n762,0,1,11,9,232,695,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,79,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n763,0,1,11,9,206,997,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n764,0,1,11,9,303,1263,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n765,0,1,9,9,9198,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,138,100,0.42,0.51,0.41,0.02,0.0,0.0,0.48,0.01\n766,0,0,5,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,76,76,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n767,0,1,11,9,345,1810,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,190,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n768,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,509,509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n769,0,1,11,9,317,258,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,185,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n770,0,2,4,9,34,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,64,59,0.92,0.03,0.02,0.0,0.0,0.0,0.0,0.0\n771,0,1,11,9,311,344,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,255,1.0,0.0,0.09,0.05,0.0,0.0,0.0,0.0\n772,0,1,11,9,224,321,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,121,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n773,0,1,11,9,273,8633,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8,255,1.0,0.0,0.12,0.03,0.0,0.0,0.0,0.0\n774,0,1,11,9,309,3222,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,253,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n775,0,1,11,9,202,4745,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n776,0,1,11,9,208,324,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,17,0.0,0.0,0.0,0.0,1.0,0.0,0.18,56,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n777,0,1,11,9,320,370,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n778,0,1,11,9,288,292,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,30,0.0,0.0,0.0,0.0,1.0,0.0,0.07,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n779,0,1,24,9,1566,331,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,164,150,0.51,0.04,0.01,0.01,0.0,0.0,0.0,0.0\n780,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n781,5,1,11,9,251,5814,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n782,0,1,11,9,224,1191,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,63,255,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n783,0,1,24,9,1593,391,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,109,179,0.83,0.04,0.01,0.01,0.01,0.0,0.0,0.0\n784,0,1,11,9,312,9558,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.75,163,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n785,0,1,11,9,216,2440,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50,255,1.0,0.0,0.02,0.03,0.02,0.01,0.0,0.0\n786,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,57,255,1.0,0.0,0.02,0.07,0.0,0.0,0.68,0.73\n787,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n788,0,1,11,9,331,8415,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,26,255,1.0,0.0,0.04,0.02,0.0,0.0,0.0,0.0\n789,0,1,11,9,155,389,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,150,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n790,0,1,11,9,232,348,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,64,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n791,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n792,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n793,0,1,11,9,228,2648,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n794,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,6,8,0.0,0.0,0.0,0.25,1.0,0.0,0.38,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05\n795,0,1,11,9,235,392,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,71,0.0,0.0,0.0,0.0,1.0,0.0,0.07,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n796,0,1,0,9,9,38,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.0,0.0,0.5,1.0,0.0,168,16,0.07,0.03,0.01,0.12,0.0,0.0,0.0,0.0\n797,0,1,11,9,236,873,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,26,0.0,0.0,0.0,0.0,1.0,0.0,0.12,217,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n798,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n799,0,1,11,9,219,1556,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,26,0.0,0.0,0.0,0.0,1.0,0.0,0.12,26,255,1.0,0.0,0.04,0.03,0.0,0.0,0.0,0.0\n800,0,1,11,9,239,1817,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n801,0,1,11,9,228,1042,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n802,0,1,11,9,220,5083,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,60,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n803,0,1,11,9,220,406,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n804,0,1,11,9,345,1515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,12,0.0,0.0,0.0,0.0,1.0,0.0,0.17,16,255,1.0,0.0,0.06,0.02,0.0,0.0,0.0,0.0\n805,0,1,11,9,322,3461,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,23,0.0,0.0,0.0,0.0,1.0,0.0,0.22,55,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n806,0,1,11,9,292,1694,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n807,0,1,11,9,301,664,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,74,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n808,0,1,11,9,221,484,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n809,0,2,4,9,45,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,93,93,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,223,0.87,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n810,0,1,11,9,454,3114,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,255,1.0,0.0,0.17,0.04,0.0,0.0,0.0,0.0\n811,0,1,11,9,221,6047,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,255,1.0,0.0,0.11,0.02,0.0,0.0,0.0,0.0\n812,0,1,11,9,243,128,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,155,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n813,0,1,9,9,2191,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,12,0.0,0.0,0.0,0.0,0.92,0.15,0.0,221,45,0.2,0.02,0.2,0.0,0.0,0.0,0.0,0.0\n814,0,1,11,9,234,13352,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n815,0,1,11,9,218,2536,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,22,22,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n816,0,0,5,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16,16,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n817,0,1,11,9,205,13599,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n818,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n819,0,1,11,9,203,3730,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,14,0.0,0.0,0.0,0.0,1.0,0.0,0.29,15,255,1.0,0.0,0.07,0.03,0.0,0.0,0.0,0.0\n820,0,1,11,9,148,4039,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n821,0,1,11,9,221,659,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,73,255,1.0,0.0,0.01,0.05,0.0,0.0,0.0,0.0\n822,0,1,11,9,374,676,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n823,0,1,11,9,299,746,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,23,255,1.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0\n824,0,1,11,9,207,654,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n825,0,1,11,9,250,2158,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,15,0.0,0.0,0.0,0.0,1.0,0.0,0.13,8,255,1.0,0.0,0.12,0.06,0.0,0.0,0.0,0.0\n826,0,1,11,9,338,1408,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n827,0,1,11,9,336,9308,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.0,0.0,0.0,1.0,0.0,0.38,1,255,1.0,0.0,1.0,0.03,0.0,0.0,0.0,0.0\n828,0,1,11,9,209,230,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,5,255,1.0,0.0,0.2,0.03,0.0,0.0,0.0,0.0\n829,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n830,0,1,11,9,235,2796,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,34,34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n831,0,1,11,9,239,302,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n832,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n833,0,1,11,9,224,10733,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n834,0,1,11,9,219,1562,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,15,0.0,0.0,0.0,0.0,1.0,0.0,0.13,30,255,1.0,0.0,0.03,0.03,0.0,0.0,0.0,0.0\n835,0,1,11,9,337,873,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n836,0,1,11,9,242,9284,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n837,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n838,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n839,0,1,11,9,282,14836,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,15,255,1.0,0.0,0.07,0.05,0.0,0.0,0.0,0.0\n840,0,1,11,9,329,272,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,243,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n841,0,1,11,9,226,3383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,47,52,0.0,0.0,0.0,0.0,1.0,0.0,0.06,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n842,0,1,11,9,164,1953,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,225,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n843,0,1,11,9,204,2367,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45,255,1.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0\n844,0,1,11,9,222,2302,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n845,0,1,24,9,1831,394,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30,167,0.8,0.1,0.03,0.01,0.0,0.0,0.0,0.0\n846,0,1,11,9,299,5733,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,63,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n847,0,1,11,9,218,280,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,24,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n848,0,2,4,9,45,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,65,47,0.72,0.03,0.02,0.0,0.0,0.0,0.0,0.0\n849,0,1,11,9,226,3431,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,190,1.0,0.0,0.03,0.06,0.0,0.0,0.0,0.0\n850,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n851,0,1,11,9,210,11297,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,59,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n852,0,1,11,9,239,4262,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n853,0,1,11,9,320,520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,37,0.0,0.0,0.0,0.0,1.0,0.0,0.16,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n854,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n855,0,1,11,9,218,17733,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,33,33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n856,0,1,11,9,283,6593,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,97,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n857,0,1,11,9,333,5834,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,55,0.0,0.0,0.0,0.0,1.0,0.0,0.09,15,255,1.0,0.0,0.07,0.02,0.0,0.0,0.0,0.0\n858,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n859,0,1,11,9,304,2507,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,254,1.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0\n860,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n861,0,1,11,9,229,511,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,27,0.0,0.0,0.0,0.0,1.0,0.0,0.11,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n862,0,1,9,9,852,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,170,89,0.47,0.04,0.47,0.02,0.0,0.0,0.01,0.0\n863,0,1,11,9,214,788,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,23,23,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n864,0,1,11,9,282,3918,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,255,1.0,0.0,0.2,0.03,0.0,0.0,0.0,0.0\n865,0,1,11,9,233,791,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n866,0,1,11,9,259,1108,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,244,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n867,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n868,0,1,11,9,206,5967,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,56,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n869,0,1,11,9,324,2015,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n870,0,1,9,9,192,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,4,0.0,0.0,0.0,0.0,1.0,0.0,0.5,109,77,0.13,0.03,0.13,0.03,0.0,0.0,0.0,0.0\n871,0,1,21,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,74,1,0.01,0.86,0.89,0.0,0.89,1.0,0.0,0.0\n872,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n873,0,1,11,9,305,1699,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,255,1.0,0.0,0.14,0.03,0.0,0.0,0.0,0.0\n874,0,1,24,9,1328,286,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,64,35,0.55,0.06,0.02,0.0,0.0,0.0,0.0,0.0\n875,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n876,0,1,11,9,309,398,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,51,51,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n877,0,1,11,9,293,2714,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n878,0,2,4,9,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,144,54,0.22,0.03,0.22,0.04,0.0,0.0,0.0,0.0\n879,0,1,11,9,210,1707,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n880,0,1,11,9,266,9710,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,48,1.0,0.0,0.09,0.04,0.0,0.0,0.0,0.0\n881,0,1,11,9,214,17469,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,178,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n882,0,1,11,9,225,7277,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,22,22,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,183,1.0,0.0,0.05,0.09,0.0,0.0,0.0,0.0\n883,0,1,11,9,235,3675,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,18,0.0,0.0,0.0,0.0,1.0,0.0,0.11,155,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n884,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n885,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n886,0,1,11,9,230,8572,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,1,255,1.0,0.0,1.0,0.07,0.0,0.0,0.0,0.0\n887,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n888,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n889,0,1,11,9,272,23907,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,36,255,1.0,0.0,0.03,0.07,0.0,0.0,0.0,0.0\n890,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,510,510,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n891,0,1,11,9,214,748,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,232,255,1.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0\n892,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n893,0,1,11,9,202,1385,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,29,0.0,0.0,0.0,0.0,1.0,0.0,0.07,28,255,1.0,0.0,0.04,0.03,0.0,0.0,0.0,0.0\n894,2,1,24,9,1844,409,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79,150,0.76,0.04,0.01,0.01,0.0,0.0,0.0,0.0\n895,0,1,11,9,235,3292,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.03,0.0,0.0,0.0,0.0\n896,0,1,11,9,191,5633,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,255,1.0,0.0,1.0,0.06,0.0,0.0,0.0,0.0\n897,0,1,11,9,233,3819,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,255,1.0,0.0,0.14,0.02,0.0,0.0,0.0,0.0\n898,0,1,11,9,275,6312,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,33,255,1.0,0.0,0.03,0.01,0.0,0.0,0.0,0.0\n899,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n900,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n901,0,1,11,9,185,751,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,55,255,1.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0\n902,0,1,11,9,252,1215,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,255,1.0,0.0,0.03,0.01,0.0,0.0,0.0,0.0\n903,0,1,11,9,244,848,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n904,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n905,0,1,24,9,1332,324,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,149,0.75,0.5,0.25,0.01,0.0,0.0,0.0,0.0\n906,0,1,11,9,247,2035,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,29,32,0.0,0.0,0.0,0.0,1.0,0.0,0.06,80,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n907,0,1,11,9,373,87461,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n908,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n909,0,1,11,9,409,9852,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13,13,0.0,0.0,0.0,0.0,1.0,0.0,0.0,41,255,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n910,0,1,11,9,286,1415,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n911,0,1,9,9,383,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,60,92,0.12,0.1,0.12,0.02,0.0,0.0,0.0,0.0\n912,0,1,11,9,223,1415,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,13,0.0,0.0,0.0,0.0,1.0,0.0,0.15,142,223,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n913,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,4,255,1.0,0.0,0.25,0.14,0.0,0.0,1.0,1.0\n914,0,2,4,9,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.75,132,72,0.3,0.04,0.3,0.03,0.0,0.0,0.0,0.0\n915,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n916,0,2,18,9,48,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22,14,0.64,0.09,0.64,0.0,0.0,0.0,0.0,0.0\n917,0,1,11,9,291,1719,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n918,0,1,11,9,233,288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20,255,1.0,0.0,0.05,0.04,0.0,0.0,0.0,0.0\n919,0,1,11,9,256,929,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n920,0,1,11,9,212,2714,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.0,0.0,52,255,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n921,0,1,11,9,227,626,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,17,0.0,0.0,0.0,0.0,1.0,0.0,0.12,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n922,0,1,26,2,126,179,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,47,47,1.0,0.0,0.02,0.0,0.04,0.04,0.96,0.96\n923,0,1,11,9,208,1969,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n924,0,1,11,9,295,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.0,0.0,1.0,0.0,0.33,166,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n925,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n926,0,1,11,9,221,1408,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n927,0,1,11,9,300,849,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,5,255,1.0,0.0,0.2,0.02,0.0,0.0,0.0,0.0\n928,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n929,0,1,11,9,263,1768,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18,21,0.0,0.0,0.0,0.0,1.0,0.0,0.14,255,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0\n930,0,1,11,9,226,11783,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n931,0,1,11,9,281,757,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,196,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n932,0,0,6,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,93,93,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n933,0,1,11,9,202,575,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n934,0,1,24,9,2077,390,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,106,148,0.58,0.03,0.01,0.01,0.01,0.0,0.0,0.0\n935,0,1,11,9,298,1099,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,10,0.0,0.0,0.0,0.0,1.0,0.0,0.3,17,255,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0\n936,0,1,11,9,326,849,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,247,1.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0\n937,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n938,0,1,11,9,203,1357,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,17,255,1.0,0.0,0.06,0.05,0.0,0.0,0.0,0.0\n939,0,1,11,9,220,16096,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,9,0.0,0.0,0.0,0.0,1.0,0.0,0.22,53,176,1.0,0.0,0.02,0.01,0.0,0.0,0.0,0.0\n940,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,33,255,1.0,0.0,0.03,0.06,0.0,0.0,1.0,1.0\n941,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n942,0,1,11,9,290,3584,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11,11,1.0,0.0,0.09,0.0,0.0,0.0,0.0,0.0\n943,0,2,4,9,52,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,81,228,1.0,0.0,1.0,0.01,0.0,0.0,0.0,0.0\n944,0,1,11,9,344,1247,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,125,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n945,0,1,11,9,348,2431,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,255,1.0,0.0,0.14,0.07,0.0,0.0,0.0,0.0\n946,0,1,11,9,147,38205,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n947,0,1,11,9,327,2402,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,28,28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n948,0,1,11,9,281,8856,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,39,0.0,0.0,0.0,0.0,1.0,0.0,0.08,11,255,1.0,0.0,0.09,0.07,0.0,0.0,0.0,0.0\n949,0,1,11,9,212,2142,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14,14,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40,255,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n950,0,1,24,9,1470,328,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,145,109,0.75,0.04,0.01,0.0,0.0,0.0,0.0,0.0\n951,0,1,11,9,234,978,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,21,0.0,0.0,0.0,0.0,1.0,0.0,0.24,12,255,1.0,0.0,0.08,0.04,0.0,0.01,0.0,0.0\n952,0,1,11,9,280,461,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,219,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n953,0,1,11,9,216,740,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,16,0.0,0.0,0.0,0.0,1.0,0.0,0.12,4,255,1.0,0.0,0.25,0.04,0.0,0.0,0.0,0.0\n954,0,1,9,9,495,0,0,0,0,0,0,1,0,0,0,5,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,16,65,0.69,0.19,0.69,0.03,0.0,0.0,0.0,0.0\n955,0,1,11,9,200,4902,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,8,0.0,0.0,0.0,0.0,1.0,0.0,0.25,8,255,1.0,0.0,0.12,0.07,0.0,0.0,0.0,0.0\n956,0,1,11,9,284,2262,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,52,255,1.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0\n957,0,1,11,9,215,6264,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,115,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n958,0,1,11,9,227,1141,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,10,0.0,0.0,0.0,0.0,1.0,0.0,0.6,3,255,1.0,0.0,0.33,0.02,0.0,0.0,0.0,0.0\n959,0,1,11,9,217,288,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,21,0.0,0.0,0.0,0.0,1.0,0.0,0.19,167,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n960,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n961,0,1,11,9,221,11200,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n962,0,1,11,9,321,608,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,10,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n963,0,1,24,9,718,334,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,0.0,0.0,0.0,0.0,1.0,0.0,0.75,160,179,0.71,0.03,0.01,0.01,0.0,0.0,0.0,0.0\n964,0,1,11,9,218,342,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,24,24,0.0,0.0,0.0,0.0,1.0,0.0,0.0,113,255,1.0,0.0,0.01,0.03,0.0,0.0,0.0,0.0\n965,0,1,11,9,323,2906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,175,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n966,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n967,0,1,11,9,332,354,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,12,0.0,0.0,0.0,0.0,1.0,0.0,0.17,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n968,0,1,9,9,26,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,86,130,0.51,0.03,0.51,0.02,0.0,0.0,0.0,0.0\n969,0,1,11,9,333,309,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,58,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n970,0,1,24,9,1397,396,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,35,240,1.0,0.0,0.03,0.02,0.0,0.0,0.0,0.0\n971,0,1,11,9,349,525,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,15,15,0.0,0.0,0.0,0.0,1.0,0.0,0.0,125,255,1.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0\n972,0,1,11,9,223,5565,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,21,0.0,0.0,0.0,0.0,1.0,0.0,0.0,21,255,1.0,0.0,0.05,0.03,0.0,0.0,0.0,0.0\n973,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n974,0,1,11,9,316,2152,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,43,0.0,0.0,0.0,0.0,1.0,0.0,0.05,10,194,1.0,0.0,0.1,0.05,0.0,0.0,0.0,0.0\n975,0,1,11,9,210,2620,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,255,1.0,0.0,0.2,0.03,0.0,0.0,0.0,0.0\n976,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n977,0,1,11,9,222,2607,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n978,0,1,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,30,2,0.03,1.0,0.03,1.0,0.0,0.0,0.9,1.0\n979,0,1,11,9,319,1471,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,10,0.0,0.0,0.0,0.0,1.0,0.0,0.2,6,255,1.0,0.0,0.17,0.05,0.0,0.02,0.0,0.0\n980,0,1,11,9,208,1180,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32,255,1.0,0.0,0.03,0.06,0.0,0.0,0.0,0.0\n981,0,1,11,9,207,935,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,72,255,1.0,0.0,0.01,0.03,0.01,0.0,0.0,0.0\n982,1,1,24,9,6093,330,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,51,129,0.53,0.08,0.02,0.02,0.0,0.0,0.0,0.0\n983,0,1,11,9,277,10067,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,28,0.0,0.0,0.0,0.0,1.0,0.0,0.11,5,255,1.0,0.0,0.2,0.02,0.0,0.0,0.0,0.0\n984,0,1,11,9,308,1155,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,176,255,1.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0\n985,0,1,11,9,322,1694,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,202,255,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0\n986,0,1,11,9,308,937,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n987,0,1,11,9,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,227,227,1.0,0.0,0.0,0.0,0.0,0.0,0.06,0.06\n988,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,123,0.48,0.01,0.48,0.0,0.0,0.0,0.0,0.0\n989,0,1,11,9,236,599,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,19,19,0.0,0.0,0.0,0.0,1.0,0.0,0.0,134,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n990,0,1,20,9,38,130,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,129,2,0.02,0.05,0.01,0.0,0.0,0.0,0.0,0.0\n991,0,1,11,9,221,1158,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,16,16,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n992,0,1,11,9,194,944,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,13,0.17,0.08,0.0,0.0,1.0,0.0,0.15,4,255,1.0,0.0,0.25,0.04,0.0,0.0,0.0,0.0\n993,0,0,5,9,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,218,218,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n994,0,1,11,9,336,1352,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,12,20,0.0,0.0,0.0,0.0,1.0,0.0,0.1,45,255,1.0,0.0,0.02,0.04,0.0,0.0,0.0,0.0\n995,0,1,11,9,205,2300,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,5,0.0,0.0,0.0,0.0,1.0,0.0,0.4,32,255,1.0,0.0,0.03,0.05,0.0,0.0,0.0,0.0\n996,0,1,11,9,267,2182,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,73,255,1.0,0.0,0.01,0.04,0.0,0.0,0.0,0.0\n997,0,1,11,9,204,516,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,11,11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n998,0,0,6,9,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0\n'
X = pd.read_csv(StringIO(X_str))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 42 columns):
Unnamed: 0                     999 non-null int64
duration                       999 non-null int64
protocol_type                  999 non-null int64
service                        999 non-null int64
flag                           999 non-null int64
src_bytes                      999 non-null int64
dst_bytes                      999 non-null int64
land                           999 non-null int64
wrong_fragment                 999 non-null int64
urgent                         999 non-null int64
hot                            999 non-null int64
num_failed_logins              999 non-null int64
logged_in                      999 non-null int64
num_compromised                999 non-null int64
root_shell                     999 non-null int64
su_attempted                   999 non-null int64
num_root                       999 non-null int64
num_file_creations             999 non-null i

In [187]:
y_str = '0,False\n1,True\n2,False\n3,False\n4,False\n5,False\n6,False\n7,False\n8,False\n9,False\n10,False\n11,True\n12,False\n13,False\n14,False\n15,False\n16,False\n17,True\n18,False\n19,True\n20,False\n21,True\n22,False\n23,True\n24,False\n25,False\n26,False\n27,False\n28,False\n29,False\n30,False\n31,False\n32,False\n33,False\n34,True\n35,False\n36,False\n37,False\n38,False\n39,False\n40,True\n41,False\n42,False\n43,False\n44,False\n45,True\n46,True\n47,False\n48,False\n49,False\n50,False\n51,True\n52,False\n53,False\n54,False\n55,False\n56,False\n57,True\n58,False\n59,False\n60,False\n61,True\n62,True\n63,True\n64,False\n65,False\n66,False\n67,False\n68,True\n69,False\n70,False\n71,False\n72,False\n73,False\n74,True\n75,False\n76,True\n77,True\n78,True\n79,True\n80,False\n81,False\n82,True\n83,True\n84,False\n85,False\n86,False\n87,False\n88,False\n89,False\n90,True\n91,False\n92,False\n93,False\n94,False\n95,False\n96,False\n97,True\n98,False\n99,False\n100,True\n101,False\n102,False\n103,False\n104,False\n105,False\n106,True\n107,False\n108,False\n109,False\n110,False\n111,False\n112,True\n113,False\n114,True\n115,False\n116,False\n117,False\n118,False\n119,False\n120,True\n121,False\n122,False\n123,False\n124,False\n125,False\n126,False\n127,False\n128,False\n129,False\n130,True\n131,False\n132,False\n133,False\n134,False\n135,True\n136,True\n137,False\n138,True\n139,False\n140,False\n141,False\n142,False\n143,False\n144,True\n145,True\n146,False\n147,False\n148,False\n149,False\n150,False\n151,False\n152,False\n153,False\n154,False\n155,True\n156,True\n157,True\n158,True\n159,False\n160,False\n161,False\n162,False\n163,False\n164,False\n165,False\n166,True\n167,False\n168,False\n169,False\n170,False\n171,False\n172,False\n173,False\n174,False\n175,False\n176,True\n177,False\n178,False\n179,False\n180,False\n181,True\n182,False\n183,False\n184,True\n185,False\n186,False\n187,False\n188,False\n189,False\n190,False\n191,False\n192,False\n193,True\n194,False\n195,False\n196,False\n197,False\n198,True\n199,True\n200,False\n201,True\n202,False\n203,True\n204,False\n205,False\n206,False\n207,False\n208,False\n209,False\n210,True\n211,False\n212,False\n213,False\n214,False\n215,False\n216,False\n217,True\n218,False\n219,True\n220,False\n221,False\n222,False\n223,False\n224,False\n225,False\n226,True\n227,True\n228,False\n229,True\n230,False\n231,False\n232,False\n233,False\n234,True\n235,False\n236,False\n237,False\n238,False\n239,False\n240,False\n241,False\n242,False\n243,False\n244,False\n245,True\n246,True\n247,False\n248,True\n249,True\n250,False\n251,True\n252,True\n253,True\n254,False\n255,True\n256,False\n257,False\n258,False\n259,True\n260,False\n261,False\n262,True\n263,True\n264,True\n265,False\n266,False\n267,False\n268,False\n269,False\n270,False\n271,False\n272,True\n273,False\n274,False\n275,True\n276,False\n277,False\n278,False\n279,True\n280,True\n281,False\n282,True\n283,False\n284,True\n285,False\n286,False\n287,False\n288,False\n289,True\n290,False\n291,False\n292,False\n293,False\n294,False\n295,False\n296,False\n297,False\n298,False\n299,False\n300,False\n301,False\n302,False\n303,False\n304,False\n305,False\n306,False\n307,False\n308,False\n309,False\n310,False\n311,False\n312,False\n313,False\n314,False\n315,False\n316,False\n317,False\n318,False\n319,False\n320,False\n321,True\n322,False\n323,False\n324,False\n325,False\n326,False\n327,False\n328,False\n329,False\n330,False\n331,False\n332,False\n333,True\n334,True\n335,True\n336,False\n337,False\n338,False\n339,False\n340,False\n341,False\n342,False\n343,True\n344,False\n345,False\n346,False\n347,False\n348,False\n349,False\n350,False\n351,False\n352,False\n353,False\n354,False\n355,False\n356,False\n357,False\n358,False\n359,False\n360,False\n361,False\n362,False\n363,False\n364,False\n365,False\n366,True\n367,True\n368,True\n369,True\n370,False\n371,False\n372,False\n373,False\n374,False\n375,False\n376,True\n377,False\n378,False\n379,True\n380,True\n381,False\n382,True\n383,False\n384,False\n385,False\n386,True\n387,False\n388,False\n389,False\n390,False\n391,True\n392,False\n393,True\n394,False\n395,False\n396,True\n397,False\n398,True\n399,False\n400,False\n401,False\n402,True\n403,True\n404,False\n405,True\n406,True\n407,False\n408,False\n409,False\n410,False\n411,False\n412,False\n413,False\n414,False\n415,False\n416,False\n417,False\n418,True\n419,False\n420,False\n421,False\n422,False\n423,False\n424,False\n425,False\n426,False\n427,False\n428,False\n429,False\n430,True\n431,False\n432,False\n433,False\n434,False\n435,False\n436,False\n437,False\n438,False\n439,False\n440,False\n441,False\n442,False\n443,False\n444,True\n445,False\n446,False\n447,True\n448,True\n449,False\n450,False\n451,False\n452,False\n453,True\n454,True\n455,False\n456,True\n457,False\n458,False\n459,False\n460,False\n461,False\n462,False\n463,False\n464,False\n465,True\n466,False\n467,False\n468,False\n469,False\n470,True\n471,False\n472,False\n473,True\n474,False\n475,False\n476,True\n477,False\n478,False\n479,True\n480,True\n481,False\n482,False\n483,False\n484,False\n485,True\n486,False\n487,True\n488,False\n489,False\n490,True\n491,False\n492,False\n493,False\n494,False\n495,False\n496,False\n497,False\n498,False\n499,False\n500,False\n501,False\n502,False\n503,True\n504,False\n505,True\n506,False\n507,True\n508,False\n509,False\n510,False\n511,False\n512,False\n513,False\n514,True\n515,False\n516,False\n517,False\n518,False\n519,True\n520,False\n521,False\n522,False\n523,False\n524,False\n525,False\n526,False\n527,False\n528,True\n529,True\n530,True\n531,False\n532,False\n533,False\n534,True\n535,False\n536,True\n537,False\n538,False\n539,True\n540,False\n541,False\n542,True\n543,False\n544,False\n545,False\n546,False\n547,False\n548,False\n549,False\n550,False\n551,False\n552,False\n553,True\n554,False\n555,False\n556,False\n557,False\n558,True\n559,True\n560,True\n561,False\n562,False\n563,False\n564,True\n565,False\n566,False\n567,False\n568,False\n569,False\n570,False\n571,False\n572,False\n573,False\n574,False\n575,True\n576,False\n577,True\n578,False\n579,True\n580,False\n581,False\n582,False\n583,True\n584,True\n585,True\n586,False\n587,False\n588,False\n589,False\n590,False\n591,True\n592,False\n593,False\n594,True\n595,False\n596,True\n597,False\n598,False\n599,False\n600,True\n601,True\n602,False\n603,False\n604,False\n605,True\n606,False\n607,False\n608,False\n609,False\n610,False\n611,False\n612,False\n613,True\n614,False\n615,True\n616,False\n617,False\n618,False\n619,False\n620,False\n621,False\n622,False\n623,False\n624,False\n625,True\n626,False\n627,False\n628,False\n629,False\n630,True\n631,False\n632,False\n633,False\n634,False\n635,False\n636,True\n637,False\n638,False\n639,True\n640,False\n641,False\n642,False\n643,False\n644,False\n645,False\n646,True\n647,False\n648,False\n649,False\n650,False\n651,False\n652,False\n653,False\n654,True\n655,False\n656,False\n657,False\n658,False\n659,True\n660,False\n661,False\n662,False\n663,False\n664,False\n665,False\n666,False\n667,True\n668,True\n669,False\n670,False\n671,False\n672,False\n673,True\n674,False\n675,False\n676,False\n677,False\n678,False\n679,False\n680,False\n681,False\n682,False\n683,False\n684,False\n685,True\n686,False\n687,False\n688,True\n689,False\n690,False\n691,True\n692,False\n693,False\n694,False\n695,False\n696,False\n697,True\n698,False\n699,False\n700,False\n701,False\n702,False\n703,False\n704,True\n705,False\n706,False\n707,True\n708,True\n709,False\n710,False\n711,False\n712,True\n713,False\n714,False\n715,True\n716,True\n717,False\n718,False\n719,False\n720,False\n721,True\n722,False\n723,True\n724,False\n725,False\n726,False\n727,False\n728,False\n729,False\n730,False\n731,True\n732,True\n733,True\n734,True\n735,False\n736,True\n737,False\n738,False\n739,False\n740,False\n741,False\n742,True\n743,False\n744,True\n745,False\n746,False\n747,False\n748,True\n749,True\n750,False\n751,False\n752,True\n753,False\n754,False\n755,True\n756,False\n757,False\n758,False\n759,False\n760,False\n761,False\n762,False\n763,False\n764,False\n765,False\n766,False\n767,False\n768,True\n769,False\n770,False\n771,False\n772,False\n773,False\n774,False\n775,False\n776,False\n777,False\n778,False\n779,False\n780,True\n781,False\n782,False\n783,False\n784,False\n785,False\n786,False\n787,True\n788,False\n789,False\n790,False\n791,True\n792,True\n793,False\n794,True\n795,False\n796,False\n797,False\n798,True\n799,False\n800,False\n801,False\n802,False\n803,False\n804,False\n805,False\n806,False\n807,False\n808,False\n809,False\n810,False\n811,False\n812,False\n813,False\n814,False\n815,False\n816,False\n817,False\n818,True\n819,False\n820,False\n821,False\n822,False\n823,False\n824,False\n825,False\n826,False\n827,False\n828,False\n829,True\n830,False\n831,False\n832,True\n833,False\n834,False\n835,False\n836,False\n837,True\n838,True\n839,False\n840,False\n841,False\n842,False\n843,False\n844,False\n845,False\n846,False\n847,False\n848,False\n849,False\n850,True\n851,False\n852,False\n853,False\n854,True\n855,False\n856,False\n857,False\n858,True\n859,False\n860,True\n861,False\n862,False\n863,False\n864,False\n865,False\n866,False\n867,True\n868,False\n869,False\n870,False\n871,True\n872,True\n873,False\n874,False\n875,True\n876,False\n877,False\n878,False\n879,False\n880,False\n881,False\n882,False\n883,False\n884,True\n885,True\n886,False\n887,True\n888,True\n889,False\n890,True\n891,False\n892,True\n893,False\n894,False\n895,False\n896,False\n897,False\n898,False\n899,True\n900,True\n901,False\n902,False\n903,False\n904,True\n905,False\n906,False\n907,False\n908,True\n909,False\n910,False\n911,False\n912,False\n913,False\n914,False\n915,True\n916,False\n917,False\n918,False\n919,False\n920,False\n921,False\n922,True\n923,False\n924,False\n925,True\n926,False\n927,False\n928,True\n929,False\n930,False\n931,False\n932,False\n933,False\n934,False\n935,False\n936,False\n937,True\n938,False\n939,False\n940,False\n941,True\n942,False\n943,False\n944,False\n945,False\n946,False\n947,False\n948,False\n949,False\n950,False\n951,False\n952,False\n953,False\n954,False\n955,False\n956,False\n957,False\n958,False\n959,False\n960,True\n961,False\n962,False\n963,False\n964,False\n965,False\n966,True\n967,False\n968,False\n969,False\n970,False\n971,False\n972,False\n973,True\n974,False\n975,False\n976,True\n977,False\n978,True\n979,False\n980,False\n981,False\n982,False\n983,False\n984,False\n985,False\n986,False\n987,True\n988,True\n989,False\n990,False\n991,False\n992,False\n993,False\n994,False\n995,False\n996,False\n997,False\n998,True\n999,False\n'
y = pd.read_csv(StringIO(y_str), index_col=0)
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 1 to 999
Data columns (total 1 columns):
False    999 non-null bool
dtypes: bool(1)
memory usage: 8.8 KB


In [188]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [190]:
# Score the test data using the given classifier
scores = clf.predict_proba(X_test)

# Get labels from the scores using the default threshold
preds = [s[1] > 0.5 for s in scores]

# Use the predict method to label the test data again
preds_default = clf.predict(X_test)

# Compare the two sets of predictions
all(preds == preds_default)

True

In [195]:
from numpy import argmax, argmin

In [198]:
# Create a range of equally spaced threshold values
t_range = [0, 0.25, 0.5, 0.75, 1.0]

# Store the predicted labels for each value of the threshold
preds = [[s[1] > thr for s in scores] for thr in t_range]

# Compute the accuracy for each threshold
accuracies = [accuracy_score(y_test, p) for p in preds]

# Compute the F1 score for each threshold
f1_scores = [f1_score(y_test, p) for p in preds]

# Report the optimal threshold for accuracy, and for F1
print(t_range[argmax(accuracies)], t_range[argmax(f1_scores)])

1.0 0


In [202]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return fp*cost_fp + fn*cost_fn

In [203]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp*fp + cost_fn*fn

# Fit a DecisionTreeClassifier to the data and compute the loss
clf = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)
print(my_scorer(y_test, clf.predict(X_test)))

570.0


In [208]:
file = 'arrh.csv'
url = 'https://assets.datacamp.com/production/repositories/3554/datasets/eb59119dbc87d95d89b446b825cb38854a59411e/'
urlretrieve(url+file, file)
arrh = pd.read_csv(file)

target = 'class'

X = arrh.drop(target, axis=1)
y = arrh[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(X_train.shape)

(339, 279)


In [218]:
# Create a scorer assigning more cost to false positives
def my_scorer(y_test, y_est, cost_fp=10.0, cost_fn=1.0):
    tn, fp, fn, tp = confusion_matrix(y_test, y_est).ravel()
    return cost_fp*fp + cost_fn*fn

# Fit a DecisionTreeClassifier to the data and compute the loss
clf = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)
print(my_scorer(y_test, clf.predict(X_test)))

# Refit with same seed, downweighting subjects weighing > 80
weights = [0.5 if w > 80 else 1.0 for w in X_train.weight]
clf_weighted = DecisionTreeClassifier(random_state=2).fit(
  X_train, y_train, sample_weight=weights)
print(my_scorer(y_test, clf_weighted.predict(X_test)))

108.0
195.0
